# Five spot reservoir using Jutul
We demonstrate a variant of a classical conceptual test case from reservoir simulation - the five-spot recovery strategy.

## Define static parameters and domain
Our first goal is to set up the grid, petrophysical properties and the time period we want to simulate.

### Define grid and time period
We begin by setting up a cartesian mesh together with a range of report steps that span five years, each made upf of 12 30 day months. The grid is 1000 by 1000 meters in the lateral direction and 50 meters in the vertical.

In [11]:
using JutulDarcy, Jutul
nx = ny = 20
nz = 2
# Some useful constants
day = 3600*24
bar = 1e5
# Define time-range for simulation
dt = repeat([30.0]*day, 12*5)
# Create mesh
dims = (nx, ny, nz)
g = CartesianMesh(dims, (1000.0, 1000.0, 50.0))

CartesianMesh((20, 20, 2), (50.0, 50.0, 25.0), [0.0, 0.0, 0.0])

### Define permeability
We create a heterogeneous permeability and porosity field from a uniformly distributed porosity together with an empirical relationship for permeability as a function of pressure. This setup probably wouldn't yield a passing grade in any geostatistical course, but it does give a pretty picture.

In [12]:
Darcy = 9.869232667160130e-13
nc = number_of_cells(g)
phi = 0.2 .+ 0.1*rand(nc)
K = @. phi^3*(1e-5)^2/(0.81*72*(1-phi)^2);

### Define the wells
We introduce four producers in each corner, and an injector in the middle of the domain using the `setup_vertical_well` helper function. At this stage, even if they have variable names that hint of their usage, we have not actually specified what these wells are going to do.

In [13]:
P_sw = setup_vertical_well(g, K, 1, 1, name = :SouthWest)
P_se = setup_vertical_well(g, K, nx, 1, name = :SouthEast)
P_ne = setup_vertical_well(g, K, 1, ny, name = :NorthEast)
P_nw = setup_vertical_well(g, K, nx, ny, name = :NorthWest)
I = setup_vertical_well(g, K, nx ÷ 2, ny ÷ 2, name = :Middle)
wells = [I, P_sw, P_se, P_ne, P_nw];

### Make a plot of our reservoir and the wells
We verify that the wells are in the expected places, and that the permeability field looks to be the right magnitude for Darcy-type flow. We load the JutulViz package that allows for plotting of both the grid and wells, using GLMakie as the backend. There is, unfortunately, some compile time latency at this step.


In [14]:
using JutulViz
fig, ax, p = plot_cell_data(g, K/Darcy)
for w in wells
    plot_well!(ax, g, w, top_factor = 0.75, textscale = 0.1)
end
fig

## Create a model and assign fluid properties
We create a two-phase immiscible fluid system made up of water (aqueous) and oileic (liquid) phases. This model is often sufficient for describing waterflooding scenarios and requires very few inputs to parametrize. We also define reference densities, that correspond to the densities of these phases once brought to *reference conditions*, which are typically at a lower pressure than in the reservoir.

In [15]:
phases = (AqueousPhase(), LiquidPhase())
sys = ImmiscibleSystem(phases)
rhoOS = 730.0
rhoWS = 1000.0
rhoS = [rhoWS, rhoOS]
model, parameters = setup_reservoir_model(g, sys, wells = wells, reference_densities = rhoS);

The code has reasonable defaults that allow a simulation to run. Let us tweak the mass density function for the phases to be more like water and a light oil phase, and set up the relative permeability variables to follow a Brooks-Corey type relationship with exponents of 2 and 3 for water and oil, respectively and corresponding residual saturations of 0.1 and 0.2. In Jutul, all local properties are called variables. These may be denoted as primary variables (unknowns) or secondary variables (determined, fundamentally, from the primary variables and other secondary variables).

In [16]:
c = [1e-6/bar, 1e-4/bar]
ρ = ConstantCompressibilityDensities(p_ref = 1*bar, density_ref = rhoS, compressibility = c)
kr = BrooksCoreyRelPerm(sys, [2.0, 3.0], [0.1, 0.2])
replace_variables!(model, PhaseMassDensities = ρ, RelativePermeabilities = kr)

MultiModel with 7 submodels:
Reservoir: ImmiscibleSystem{Tuple{AqueousPhase, LiquidPhase}} ∈ MinimalTPFAGrid{Float64, Int64}
Middle: ImmiscibleSystem{Tuple{AqueousPhase, LiquidPhase}} ∈ MultiSegmentWell
SouthWest: ImmiscibleSystem{Tuple{AqueousPhase, LiquidPhase}} ∈ MultiSegmentWell
SouthEast: ImmiscibleSystem{Tuple{AqueousPhase, LiquidPhase}} ∈ MultiSegmentWell
NorthEast: ImmiscibleSystem{Tuple{AqueousPhase, LiquidPhase}} ∈ MultiSegmentWell
NorthWest: ImmiscibleSystem{Tuple{AqueousPhase, LiquidPhase}} ∈ MultiSegmentWell
Facility: PredictionMode ∈ WellGroup


## Define the operational schedule of the wells
Our reservoir, wells and fluid model is now set up, but nothing much will happen if we were to simulate the current configuration. It is time to introduce a few operational controls to the wells. We let the middle well operate as an injector that injects a pure stream of water, corresponding to a full pore-volume at reference conditions injected over the full time period. The producers operate at fixed bottom hole pressure.

Jutul supports advanced on limits on each well, and different controls for each report time step. Here, we make the simplified assumption that all wells operate at the same controls throughout the five years, and that the default limits that prevent producers from turning into injectors are sufficient to keep the scenario realistic.

In [17]:
reservoir = reservoir_model(model);
pv = pore_volume(reservoir)
inj_rate = sum(pv)/sum(dt)
rate_target = TotalRateTarget(inj_rate)
I_ctrl = InjectorControl(rate_target, [1.0, 0.0], density = rhoWS)
# Define function to make bhp controls easy to setup
bh_well(p) = ProducerControl(BottomHolePressureTarget(p*bar))

controls = Dict()
controls[:Middle] = I_ctrl
controls[:SouthWest] = bh_well(50.0)
controls[:SouthEast] = bh_well(55.0)
controls[:NorthEast] = bh_well(45.0)
controls[:NorthWest] = bh_well(75.0)

ProducerControl{BottomHolePressureTarget}(BottomHolePressureTarget(7.5e6))

## Perform a simulation

### Define forces and initial state of the reservoir
We assume a green field with no connate water, and let the reservoir be completely filled with oil.

In [18]:
forces = setup_reservoir_forces(model, control = controls)
state0 = setup_reservoir_state(model, Pressure = 150*bar, Saturations = [0.0, 1.0]);

### Perform the simulation
We set up a defaulted reservoir simulator and perform a simulation. We disable most output by setting a low info level since we are using a notebook and not the REPL, but enable the end report to see a breakdown of nonlinear iterations and time taken.

In [19]:
sim, config = setup_reservoir_simulator(model, state0, parameters, info_level = -1, end_report = true)
states, reports = simulate!(sim, dt, forces = forces, config = config);

Number of iterations
╭────────────────┬──────────┬──────────────┬──────────────┬────────┬───────╮
│ Type           │ Avg/step │ Avg/ministep │     Time per │ Wasted │ Total │
│                │ 60 steps │ 67 ministeps │ Milliseconds │        │       │
├────────────────┼──────────┼──────────────┼──────────────┼────────┼───────┤
│ Newtons        │  4.01667 │      3.59701 │       4.6415 │      0 │   241 │
│ Linearizations │  5.13333 │      4.59701 │       3.6318 │      0 │   308 │
╰────────────────┴──────────┴──────────────┴──────────────┴────────┴───────╯
Simulator timing
╭──────────────┬──────────────┬──────────┬─────────╮
│ Name         │         Each │ Fraction │   Total │
│              │ Milliseconds │  Percent │ Seconds │
├──────────────┼──────────────┼──────────┼─────────┤
│ Properties   │       1.2694 │  34.95 % │  0.3910 │
│ Assembly     │       0.4008 │  11.04 % │  0.1234 │
│ Linear solve │       0.9975 │  21.49 % │  0.2404 │
│ Update       │       0.6787 │  14.62 % │  0.1636 │

┌ Warning: Linear solver: Breakdown bᵀc = 0, final residual: 11.515121475799896, rel. value 1.0. rtol = 0.005, atol = 0.0, max_it = 200, solver = bicgstab
└ @ Jutul C:\Users\olavm\.julia\dev\Jutul\src\linsolve\krylov.jl:145


### Plot the results

In [20]:
f, = plot_interactive(g, map(x -> x[:Reservoir], states))
display(f)

GLMakie.Screen(...)